In [1]:
import numpy as np
import pandas as pd
import os, re, math, platform
from pathlib import Path
import matplotlib.pyplot as plt
import json
import joblib
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from sklearn.metrics import precision_recall_curve, roc_curve, auc, fbeta_score
from imblearn.metrics import geometric_mean_score
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier 
from xgboost import plot_importance
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis as PA
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor
from matplotlib import pyplot
from sklearn.metrics import matthews_corrcoef, confusion_matrix,precision_recall_curve, roc_curve, auc, fbeta_score,roc_auc_score
from fea_extract import read_fasta,insert_AAC,insert_DPC,insert_CKSAAGP,insert_CTD,insert_PAAC,insert_AAI,insert_GTPC,insert_QSO,insert_AAE,insert_PSAAC,insert_word2int,insert_ASDC
import warnings 
from collections import Counter
from tools import cv,evaluate
import matplotlib.pyplot as plt
import seaborn as sns
Path('./results/balance_eval_AUC/').mkdir(exist_ok=True,parents=True)
warnings.filterwarnings('ignore')
seed=10

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "7";

In [3]:
#!pip install lightgbm
from sklearn.ensemble import BaggingClassifier

In [4]:
def pro_data(seq):
    df_n = insert_PAAC(seq)
    df_n = insert_AAC(df_n)
    df_n = insert_CKSAAGP(df_n)
    df_n = insert_CTD(df_n)
    df_n = insert_DPC(df_n)
    df_n = insert_GTPC(df_n)
    df_n = insert_QSO(df_n)
    df_n = insert_AAE(df_n)
    df_n = insert_ASDC(df_n)
    #df_n = insert_word2int(df_n)
    return df_n

In [5]:
seq_X_train = pd.read_csv('data/train/X_train.csv')
seq_X_test = pd.read_csv('data/test/X_test.csv')
seq_y_train = pd.read_csv('data/train/y_train.csv')
seq_y_test = pd.read_csv('data/test/y_test.csv')

In [6]:
Seq_X_train = pro_data(seq_X_train)
Seq_X_test = pro_data(seq_X_test)

In [7]:
Seq_X_train

,Id,Sequence,Xc1.A,Xc1.R,Xc1.N,Xc1.D,Xc1.C,Xc1.Q,Xc1.E,Xc1.G,...,ASDC_390,ASDC_391,ASDC_392,ASDC_393,ASDC_394,ASDC_395,ASDC_396,ASDC_397,ASDC_398,ASDC_399
0,Sequence_107,FLPGLIAGIAKML,0.735496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.735496,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Sequence_559,FAKLLAKLAKAKG,1.012730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.253182,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,seq_1418,VILMFVKKKAAKPYRQVLRNIYRKWRPKIR,0.439350,1.098376,0.219675,0.000000,0.000000,0.219675,0.000000,0.000000,...,0.0,0.002299,0.004598,0.002299,0.018391,0.000000,0.000000,0.002299,0.004598,0.002299
3,Sequence_445,GIGKFLHSAKKFGKAFVGEIMNS,0.513077,0.000000,0.256539,0.000000,0.000000,0.000000,0.256539,1.026155,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Sequence_282,FAKLLAKLAK,0.752074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,Sequence_86,RIITCSCRTFCFLGERISGRCYQSVFIYRLCCRG,0.000000,1.385464,0.000000,0.000000,1.385464,0.230911,0.230911,0.692732,...,0.0,0.000000,0.000000,0.001783,0.007130,0.001783,0.000000,0.001783,0.000000,0.001783
1965,seq_257,VLLNKARLAGFNTLRNCYRSNFKKHYHKTC,0.520853,0.781280,1.041707,0.000000,0.520853,0.000000,0.000000,0.260427,...,0.0,0.002299,0.000000,0.000000,0.002299,0.002299,0.004598,0.000000,0.000000,0.002299
1966,Sequence_628,IFGAIAGLLKNIF,0.673935,0.000000,0.336968,0.000000,0.000000,0.000000,0.000000,0.673935,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1967,Sequence_329,KDRPKKPGLCPPRPQKPCVKECKNDWSCPGQQKCCNYGCIDECRDP...,0.000000,0.968983,0.968983,1.291977,2.583954,0.968983,0.645989,0.968983,...,0.0,0.000816,0.000816,0.000000,0.000816,0.000000,0.000000,0.000816,0.000000,0.000000


In [8]:
Seq_X_test

,Id,Sequence,Xc1.A,Xc1.R,Xc1.N,Xc1.D,Xc1.C,Xc1.Q,Xc1.E,Xc1.G,...,ASDC_390,ASDC_391,ASDC_392,ASDC_393,ASDC_394,ASDC_395,ASDC_396,ASDC_397,ASDC_398,ASDC_399
0,seq_1723,VILIFVKKKAAKPLRQVYRNIYRHWRPKWW,0.453280,0.906559,0.226640,0.000000,0.000000,0.226640,0.000000,0.000000,...,0.000000,0.002299,0.004598,0.000000,0.011494,0.000000,0.000000,0.000000,0.013793,0.002299
1,seq_1691,VMLIFVKKKAAKPLRQVLRNIYRHWRPKIW,0.454513,0.909026,0.227257,0.000000,0.000000,0.227257,0.000000,0.000000,...,0.000000,0.000000,0.002299,0.000000,0.004598,0.000000,0.000000,0.000000,0.004598,0.000000
2,seq_1177,VMLIFVKKKAAKPYRQVYRNIYRHMRPKWW,0.468333,0.936666,0.234166,0.000000,0.000000,0.234166,0.000000,0.000000,...,0.006897,0.004598,0.006897,0.002299,0.020690,0.000000,0.000000,0.002299,0.013793,0.006897
3,seq_1411,VILIFVKKKAAKPMRQVLRNIYRHWRPKYK,0.475549,0.951098,0.237775,0.000000,0.000000,0.237775,0.000000,0.000000,...,0.000000,0.000000,0.002299,0.000000,0.004598,0.000000,0.000000,0.000000,0.002299,0.002299
4,seq_1312,VILIFVKKKAAKPLRKVLRNIYRHWRPKWQ,0.450858,0.901715,0.225429,0.000000,0.000000,0.225429,0.000000,0.000000,...,0.000000,0.000000,0.002299,0.002299,0.004598,0.000000,0.000000,0.000000,0.004598,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,Sequence_82,VIPFVASVAAEMMPHVYCAASRKC,1.854726,0.370945,0.000000,0.000000,0.741890,0.000000,0.370945,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003623,0.003623,0.000000,0.000000,0.000000,0.000000
489,Sequence_610,TTPVCAVAATAAASSAACGWVGGGIFTGVTVVVSLKHC,3.697450,0.000000,0.000000,0.000000,1.386544,0.000000,0.000000,2.310906,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
490,Sequence_543,GMSGYIQGIPDFLKGYLHGISAANKHKKGRL,0.589752,0.294876,0.294876,0.294876,0.000000,0.294876,0.000000,1.769255,...,0.000000,0.004301,0.002151,0.002151,0.004301,0.004301,0.000000,0.000000,0.000000,0.002151
491,Sequence_19,RCVCTRGFCRCVCTRGFC,0.000000,0.810645,0.000000,0.000000,1.215967,0.000000,0.000000,0.405322,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
Seq_X_train.to_csv('data/train/Seq_X_train_all.csv',index=False)
Seq_X_test.to_csv('data/test/Seq_X_test_all.csv',index=False)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier

from sklearn.svm import LinearSVC

ml = ["DT","LGBM","XGBoost","RF","ET","GBDT"]

#,"SVM","MLP","LR","AB","voting_clf","stacking","Two_eclf"
layer_one= [  LGBMClassifier(random_state = seed),ExtraTreesClassifier(random_state=seed),RandomForestClassifier(random_state=seed),MLPClassifier(hidden_layer_sizes=700,learning_rate="adaptive",random_state=seed)]
                     #('rf5',GaussianNB()),
#                   ('SVM', SVC(kernel='linear',probability=True))]
layer_two = [SVC(random_state=seed,probability=True), ExtraTreesClassifier(random_state=seed),RandomForestClassifier(random_state=seed),LGBMClassifier(random_state = seed)]
             #GaussianNB()]
#                   ('SVC', SVC(kernel='linear',probability=True))]

layer_two_meta= StackingClassifier(classifiers = layer_one, meta_classifier=LogisticRegression())    
DT  = DecisionTreeClassifier(random_state=seed)
LGBM = LGBMClassifier(random_state = seed)
GBDT = GradientBoostingClassifier(random_state=seed)
ET = ExtraTreesClassifier(random_state=seed)
SVM = SVC(random_state=seed,probability=True)
MLP = MLPClassifier(hidden_layer_sizes=700,learning_rate="adaptive",random_state=seed)
RF = RandomForestClassifier(random_state=seed)
XGBoost = XGBClassifier(random_state=seed)
LR  = LogisticRegression(random_state=seed,solver='liblinear')
AB = AdaBoostClassifier(random_state=seed)
#SDG_Classifier = SGDClassifier(max_iter=1000, tol=1e-3) 
voting_clf = VotingClassifier(estimators = [('lgbm',LGBM),('rf',RF),('xgb',XGBoost),('ET',ET),('GBDT',GBDT)], voting = 'soft')
#Bagging_clf = BaggingClassifier(base_estimator=RandomForestClassifier(random_state=seed))
#VOT_STACK = StackingClassifier(classifiers = voting_clf, meta_classifier=LR)
stacking = StackingClassifier(classifiers=[LGBM,RF,XGBoost,ET],meta_classifier=LR)
Two_eclf = StackingClassifier(classifiers=layer_two, meta_classifier=layer_two_meta)

In [11]:
fea = Seq_X_train.columns[2:]
X_train = Seq_X_train[fea].to_numpy()
X_test = Seq_X_test[fea].to_numpy()
y_train = seq_y_train.to_numpy()
y_test = seq_y_test.to_numpy()

In [12]:
len(fea)

1296

In [13]:
index = []
ALL_eval=pd.DataFrame()
for i in ml:
    print('process_{}'.format(i))
    model = eval(i)
    Evals = cv(model,X_train,y_train)
    ALL_eval = pd.concat([ALL_eval,Evals],axis=1)
    index.append("{}".format(i))
ALL_eval.columns = index

process_DT
process_LGBM
[LightGBM] [Info] Number of positive: 873, number of negative: 702
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047458 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76429
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 1157
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.554286 -> initscore=0.218002
[LightGBM] [Info] Start training from score 0.218002
[LightGBM] [Info] Number of positive: 878, number of negative: 697
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75075
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 1153
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557460 -> initscore=0.230861
[LightGBM] [Info] Start training from score

In [14]:
ALL_eval

,DT,LGBM,XGBoost,RF,ET,GBDT
ACC,0.823262,0.866439,0.864408,0.866432,0.866434,0.854751
F1,0.841541,0.878572,0.875718,0.875903,0.875792,0.868040
F2,0.842787,0.873558,0.866320,0.860294,0.859099,0.863307
GMean,0.820186,0.865917,0.864809,0.868225,0.868417,0.854134
SEN,0.843692,0.870370,0.860401,0.850426,0.848501,0.860298
PREC,0.839747,0.887605,0.892964,0.904412,0.906009,0.876577
SPEC,0.797767,0.861937,0.870151,0.887263,0.889486,0.848448
MCC,0.642025,0.731078,0.728595,0.734557,0.734555,0.707434
AUC,0.820729,0.949818,0.949781,0.949154,0.949955,0.943224
AUPR,0.795412,0.964579,0.964519,0.963549,0.964205,0.959913


In [15]:
index = []
ALL_eval_test=pd.DataFrame()
for i in ml:
    eval_dict = []
    print('process_{}'.format(i))
    model = eval(i)
    model.fit(X_train,y_train)
    eval_dictionary = evaluate(X_test,y_test,model)
    eval_dict = eval_dict+[eval_dictionary]
    Evals = pd.DataFrame(eval_dict).T
    ALL_eval_test = pd.concat([ALL_eval_test,Evals],axis=1)
    index.append("{}".format(i))
ALL_eval_test.columns = index

process_DT
process_LGBM
[LightGBM] [Info] Number of positive: 1094, number of negative: 875
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057729 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85021
[LightGBM] [Info] Number of data points in the train set: 1969, number of used features: 1190
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.555612 -> initscore=0.223372
[LightGBM] [Info] Start training from score 0.223372
process_XGBoost
process_RF
process_ET
process_GBDT


In [ ]:
evals = ALL_eval.loc[['AUC'],:]
sns.set()
evals.T.plot.bar(figsize=(7,5),rot=45,width = 0.7,grid = False)
plt.ylim(0.6,1.0)
plt.ylabel('Area Under Curve',fontsize=18)
plt.xlabel('Performance of ML',fontsize=18)
plt.legend(loc =  'lower left' )

In [ ]:
ALL_eval_test

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming ALL_eval_test is a DataFrame with metrics as columns and models as rows

# Create a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(ALL_eval_test, cmap='coolwarm', annot=True, fmt=".2f", cbar=True)

plt.title('Evaluation Metrics Heatmap_independent_test', fontsize=18)
plt.xlabel('MODEL', fontsize=14)
plt.ylabel('METRICS',size=14)

plt.show()

In [ ]:
# this is a code for evaluation >>effects after data normalization and standarization </h1>
#Scaling the train and test dataset for training
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
nms = MinMaxScaler()
X_train_norm = nms.fit_transform(X_train)
X_test_norm = nms.transform(X_test)
#标准化
stdsc = StandardScaler(with_mean=False)
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)


In [ ]:
#Save the standarzied and normalixed data
X_train_norm1 = pd.DataFrame(X_train_norm)
X_train_norm1.to_csv('data/train/X_train_norm.csv',index=False)
X_test_norm1 = pd.DataFrame(X_test_norm)
X_test_norm1.to_csv('data/test/X_test_norm.csv',index=False)
#standarization
X_train_std1 = pd.DataFrame(X_train_std)
X_train_std1.to_csv('data/train/X_train_std.csv',index=False)
X_test_std1 = pd.DataFrame(X_test_std)
X_test_std1.to_csv('data/test/X_test_std.csv',index=False)

In [ ]:
index = []
ALL_eval=pd.DataFrame()
for i in ml:
    print('process_{}'.format(i))
    model = eval(i)
    Evals = cv(model,X_train_norm,y_train)
    ALL_eval = pd.concat([ALL_eval,Evals],axis=1)
    index.append("{}".format(i))
ALL_eval.columns = index

In [ ]:
ALL_eval

In [ ]:
index = []
ALL_eval=pd.DataFrame()
for i in ml:
    print('process_{}'.format(i))
    model = eval(i)
    Evals = cv(model,X_train_std,y_train)
    ALL_eval = pd.concat([ALL_eval,Evals],axis=1)
    index.append("{}".format(i))
ALL_eval.columns = index

In [ ]:
#FIVE-FOLD RESULT FOR TRAINING DATASET
ALL_eval

In [ ]:
evals = ALL_eval.loc[['AUC'],:]
sns.set()
evals.T.plot.bar(figsize=(7,5),rot=45,width = 0.7,grid = False)
plt.ylim(0.6,1.0)
plt.ylabel('Area Under Curve',fontsize=18)
plt.xlabel('Performance of ML',fontsize=18)
plt.legend(loc =  'lower left' )

In [ ]:
index = []
ALL_eval_test=pd.DataFrame()
for i in ml:
    eval_dict = []
    print('process_{}'.format(i))
    model = eval(i)
    model.fit(X_train_norm,y_train)
    eval_dictionary = evaluate(X_test_norm,y_test,model)
    eval_dict = eval_dict+[eval_dictionary]
    Evals = pd.DataFrame(eval_dict).T
    ALL_eval_test = pd.concat([ALL_eval_test,Evals],axis=1)
    index.append("{}".format(i))
ALL_eval_test.columns = index

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming ALL_eval_test contains accuracy and MCC values for different models

# Generate a heat map for accuracy and MCC side by side
fig, axes = plt.subplots(1, 2, figsize=(15, 8))

# Accuracy Heat Map
sns.heatmap(ALL_eval_test.loc[['ACC']], annot=True, cmap="YlGnBu", ax=axes[0])
axes[0].set_title('Accuracy Heat Map for validation dataset AntiCP 2.0')

# MCC Heat Map
sns.heatmap(ALL_eval_test.loc[['MCC']], annot=True, cmap="YlGnBu", ax=axes[1])
axes[1].set_title('MCC Heat Map for validation dataset AntiCP 2.0')

plt.tight_layout()
plt.show()

In [ ]:
ALL_eval_test